In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB 

# Building a SMS spam detector

In [ ]:
# load the dataset
url = 'https://raw.githubusercontent.com/um-perez-alvaro/Data-Science-Practice/master/Data/sms.tsv.txt'
sms = pd.read_csv(url, sep='\t', header=None, names=['label', 'message'])

In [ ]:
sms.head()

In [ ]:
# spam example
print(sms[sms.label=='spam'].message.iloc[100])

In [ ]:
# ham example
print(sms[sms.label=='ham'].message.iloc[100])

In [ ]:
sms.label.value_counts()

In [ ]:
# feature matrix/target vector
X = sms.message
y = sms.label

In [ ]:
# train/test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y)

In [ ]:
# initialize the vectorizer (with default parameters)
vect = CountVectorizer(stop_words='english',max_features=1000,min_df=10)

In [ ]:
# learn training vocabulary, then use it to create a document-term matrix
vect.fit(X_train)
X_train_dtm = vect.transform(X_train)

In [ ]:
# transform testing data (using fitted vocabulary) into a document-term matrix
X_test_dtm = vect.transform(X_test)

## Naive Bayes model

In [ ]:
# import and initialize a Multinomial Naive Bayes model
from sklearn.naive_bayes import MultinomialNB
nb_clf = MultinomialNB()

In [ ]:
# train the model using X_train_dtm 
nb_clf.fit(X_train_dtm, y_train)

In [ ]:
# make class predictions for X_test_dtm
y_test_pred = nb_clf.predict(X_test_dtm)

In [ ]:
# evaluate the model
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
# accuracy
accuracy_score(list(y_test), y_test_pred)

In [ ]:
# confusion matrix
confusion_matrix(y_test, y_test_pred)

In [ ]:
# print messages text for the false positives (ham incorrectly classified as spam) 
X_test[(y_test=='ham') & (y_test_pred=='spam')]

In [ ]:
X_test[3797]

In [ ]:
# print messages text for the false negatives (span incorrectly classified as ham)  
X_test[(y_test=='spam') & (y_test_pred=='ham')]

In [ ]:
# example of false negatives
X_test[4514]

## How does Naive Bayes choose between spam and ham

In [ ]:
# store the vocabulary of X_train
words = vect.get_feature_names()

In [ ]:
len(words)

In [ ]:
# Naive Bayes counts the number of times each word appears in each class
# Rows represent classes (ham and spam), columns represent words
nb_clf.feature_count_

In [ ]:
nb_clf.classes_

In [ ]:
# number of times each word appears across all ham messages
ham_word_count = nb_clf.feature_count_[0,:]
# number of times each word appears across all spam messages
spam_word_count = nb_clf.feature_count_[1,:]

In [ ]:
# create a DataFrame of words with their separate ham and spam counts
words = pd.DataFrame({'word' : words, 'ham' : ham_word_count, 'spam' : spam_word_count}).set_index('word')
words.head()

In [ ]:
# add 1 to the columns counts to avoid dividing by 0
words.ham = words.ham+1
words.spam = words.spam+1

In [ ]:
# convert the ham and spam counts into frequencies
words.ham = words.ham/words.ham.sum()
words.spam = words.spam/words.spam.sum()
words.head()

In [ ]:
# calculate the ratio of ham-to-spam and spam-to-ham for each word
words['ham_ratio'] = words.ham/words.spam
words['spam_ratio'] = words.spam/words.ham

In [ ]:
# top 20 spam words
words.sort_values(by='spam_ratio', ascending=False).head(5)

In [ ]:
# top 20 ham words
words.sort_values(by='ham_ratio', ascending=False).head(5)

## Logistic Regression Model

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
log_clf = LogisticRegression()

In [ ]:
log_clf.fit(X_train_dtm,y_train)

In [ ]:
y_test_pred = log_clf.predict(X_test_dtm)

In [ ]:
accuracy_score(y_test,y_test_pred)

In [ ]:
confusion_matrix(y_test,y_test_pred)

In [ ]:
print(X_test[(y_test == 'spam') & (y_test_pred == 'ham')].iloc[5])

In [ ]:
# top coefficients
coeffs = pd.DataFrame(data = log_clf.coef_.T, index=vect.get_feature_names(),columns=['coefficient'])
coeffs.sort_values(by='coefficient').tail(20).plot.barh(figsize=(5,10))

In [ ]:
log_clf.classes_

# From occurrences to frequencies

Occurrence count is a good start but there is an issue: longer documents will have higher average count values than shorter documents, even though they might talk about the same topics.

To avoid these potential discrepancies it suffices to divide the number of occurrences of each word in a document by the total number of words in the document: these new features are called **tf (for Term Frequencies)**.

Another refinement on top of tf is to downscale weights for words that occur in many documents in the corpus and are therefore less informative than those that occur only in a smaller portion of the corpus. This downscaling is called **tf–idf (for “Term Frequency times Inverse Document Frequency”)**.

Both tf and tf–idf can be computed using [TfidfTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html#sklearn.feature_extraction.text.TfidfTransformer)

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
# initialize
tf_transformer = TfidfTransformer(use_idf=False) # use tf
# fit
tf_transformer.fit(X_train_dtm)
# transform
X_train_tf = tf_transformer.transform(X_train_dtm)
X_test_tf = tf_transformer.transform(X_test_dtm)

In [ ]:
log_clf.fit(X_train_tf,y_train)
y_test_pred = log_clf.predict(X_test_tf)

In [ ]:
accuracy_score(y_test,y_test_pred)

In [ ]:
confusion_matrix(y_test,y_test_pred)